<a href="https://colab.research.google.com/github/moshe-hadad/knesset-network/blob/main/knessset_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing needed libraries


In [ ]:
# Run this in first run to install pyvis for the graph display
# !pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.6 MB/s eta 0:00:00


# Setting constants  variables for the links to the data sets on GitHub

In [ ]:

FACTION_URL ='https://raw.githubusercontent.com/moshe-hadad/knesset-network/main/data/factions.csv'
FACTION_PEOPLE_URL = 'https://raw.githubusercontent.com/moshe-hadad/knesset-network/main/data/faction_people.csv'
PERSONS_URL = 'https://raw.githubusercontent.com/moshe-hadad/knesset-network/main/data/KNS_Person_data.csv'


,FactionID,Name,KnessetNum,StartDate,EndDate,IsCurrent,NumberOfMembers,StatusID,StatusDesc,LastUpdatedDate
0,3,ישראל אחת,15,1999-06-07T00:00:00,NaN,False,NaN,NaN,NaN,2019-01-24T11:45:06.46
1,4,הליכוד,15,1999-06-07T00:00:00,NaN,False,NaN,NaN,NaN,2019-01-24T11:45:06.46
2,5,"ש""ס",15,1999-06-07T00:00:00,NaN,False,NaN,NaN,NaN,2019-01-24T11:45:06.46
3,6,מרצ,15,1999-06-07T00:00:00,NaN,False,NaN,NaN,NaN,2019-01-24T11:45:06.46
4,7,האיחוד הלאומי - ישראל ביתנו,15,2000-02-01T00:00:00,NaN,False,NaN,NaN,NaN,2022-01-23T15:10:00


# Remotly loading data sets and presenting some first rows

In [ ]:
import pandas as pd
factions_df = pd.read_csv(FACTION_URL)
factions_df.head()

In [ ]:

factions_ppl_df = pd.read_csv(FACTION_PEOPLE_URL)
factions_ppl_df.head()

,PersonID,KnessetNum,FactionID
0,4217,17,638
1,4302,17,634
2,528,16,424
3,526,16,422
4,549,16,52


In [ ]:
persons_df = pd.read_csv(PERSONS_URL)
persons_df.head()

In [ ]:
from pyvis import network
import networkx as nx
from IPython.display import display, HTML

# Create a sample graph
G = nx.karate_club_graph()

# Create a network object with inline resources
net = network.Network(notebook=True, cdn_resources='remote')

# Load the NetworkX graph into the network object
net.from_nx(G)

# Show the network
net.save_graph("networkx-pyvis.html")
HTML(filename="networkx-pyvis.html")
# net.show("karate_club_graph.html")